In [108]:
# Importing the necesary libraries that we are going to work with.

import pandas as pd
import yaml
from riotwatcher import LolWatcher, ApiError

In [126]:
config = yaml.load(open("riot-config.yaml"), Loader=yaml.FullLoader) # Reading in the riot-config.yaml file

lol_watcher = LolWatcher(config['API']) # Riot API Key

region = 'eun1' # Defining the region of the accounts scraped.

player_info = lol_watcher.summoner.by_name(region, 'xzxxsx') # Get player's info by their IGN. 

player_ranked_stats = lol_watcher.league.by_summoner(region, player_info['id']) # Get player's ranked stats.

player_matches = lol_watcher.match.matchlist_by_account(my_region, me['accountId']) # Get player's match history. Limited to last 100 records.

champ_list = lol_watcher.data_dragon.champions("10.15.1", False, 'en_US')['data'] # Get all available info about an in-game champion.

# For Riot's API, the 404 status code indicates that the requested data wasn't found and
# should be expected to occur in normal operation, as in the case of a an
# invalid summoner name, match ID, etc.
#
# The 429 status code indicates that the user has sent too many requests
# in a given amount of time ("rate limiting").

# Custom Error:
# try:
#     response = lol_watcher.summoner.by_name(region, 'this_is_probably_not_anyones_summoner_name')
# except ApiError as err:
#     if err.response.status_code == 429:
#         print('We should retry in {} seconds.'.format(err.response.headers['Retry-After']))
#         print('this retry-after is handled by default by the RiotWatcher library')
#         print('future requests wait until the retry-after time passes')
#     elif err.response.status_code == 404:
#         print('Summoner with that ridiculous name not found.')
#     else:
#         raise

In [131]:
player_ranked_stats

[{'leagueId': 'e1f24cd8-e022-43ea-afa8-c08153613ace',
  'queueType': 'RANKED_FLEX_SR',
  'tier': 'BRONZE',
  'rank': 'I',
  'summonerId': '5kbw3hTPLFJKW-85vpzyte9QspyLT0oswOfE2406yR_oHGw',
  'summonerName': 'xzxxsx',
  'leaguePoints': 74,
  'wins': 17,
  'losses': 19,
  'veteran': False,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': False}]

In [124]:
# fetch match details
last_match = player_matches['matches'][0]
match_detail = lol_watcher.match.by_id(my_region, last_match['gameId'])
participants = []
for row in match_detail['participants']:
    participants_row = {}
    participants_row['gameID'] = last_match['gameId']
    participants_row['champion'] = row['championId']
    participants_row['spell1'] = row['spell1Id']
    participants_row['spell2'] = row['spell2Id']
    participants_row['win'] = row['stats']['win']
    participants_row['kills'] = row['stats']['kills']
    participants_row['deaths'] = row['stats']['deaths']
    participants_row['assists'] = row['stats']['assists']
    participants_row['totalDamageDealt'] = row['stats']['totalDamageDealt']
    participants_row['goldEarned'] = row['stats']['goldEarned']
    participants_row['champLevel'] = row['stats']['champLevel']
    participants_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
    participants_row['item0'] = row['stats']['item0']
    participants_row['item1'] = row['stats']['item1']
    participants.append(participants_row)
df = pd.DataFrame(participants)
df

,gameID,champion,spell1,spell2,win,kills,deaths,assists,totalDamageDealt,goldEarned,champLevel,totalMinionsKilled,item0,item1
0,2542786996,18,4,7,True,8,6,3,165814,12727,16,181,3095,3156
1,2542786996,53,4,14,True,3,5,11,15876,7780,15,10,3855,3109
2,2542786996,50,14,4,True,8,6,7,154657,14149,17,196,1056,3157
3,2542786996,141,4,11,True,10,5,0,231189,13710,17,52,3142,3812
4,2542786996,17,4,12,True,1,7,4,188174,11688,17,222,3151,3115
5,2542786996,202,4,21,False,3,6,9,150545,11373,15,208,3094,3009
6,2542786996,67,4,14,False,5,4,4,116363,11686,17,191,1054,3153
7,2542786996,29,4,11,False,9,4,7,160412,13266,17,37,1416,3153
8,2542786996,54,3,4,False,3,9,9,33173,7957,13,46,3860,3285
9,2542786996,105,4,14,False,9,7,5,116276,12433,15,180,1052,3152


In [127]:
# check league's latest version
# latest = lol_watcher.data_dragon.versions_for_region(my_region)['n']['champion']
# Lets get some champions static information
# static_champ_list = lol_watcher.data_dragon.champions("10.15.1", False, 'en_US')
# champ static list data to dict for looking up
champ_dict = {}
for key in champ_list:
    row = champ_list[key]
    champ_dict[row['key']] = row['id']
for row in participants:
    print(str(row['champion']) + ' ' + champ_dict[str(row['champion'])])
    row['championName'] = champ_dict[str(row['champion'])]
# print dataframe
df = pd.DataFrame(participants)
df

18 Tristana
53 Blitzcrank
50 Swain
141 Kayn
17 Teemo
202 Jhin
67 Vayne
29 Twitch
54 Malphite
105 Fizz


,gameID,champion,spell1,spell2,win,kills,deaths,assists,totalDamageDealt,goldEarned,champLevel,totalMinionsKilled,item0,item1,championName
0,2542786996,18,4,7,True,8,6,3,165814,12727,16,181,3095,3156,Tristana
1,2542786996,53,4,14,True,3,5,11,15876,7780,15,10,3855,3109,Blitzcrank
2,2542786996,50,14,4,True,8,6,7,154657,14149,17,196,1056,3157,Swain
3,2542786996,141,4,11,True,10,5,0,231189,13710,17,52,3142,3812,Kayn
4,2542786996,17,4,12,True,1,7,4,188174,11688,17,222,3151,3115,Teemo
5,2542786996,202,4,21,False,3,6,9,150545,11373,15,208,3094,3009,Jhin
6,2542786996,67,4,14,False,5,4,4,116363,11686,17,191,1054,3153,Vayne
7,2542786996,29,4,11,False,9,4,7,160412,13266,17,37,1416,3153,Twitch
8,2542786996,54,3,4,False,3,9,9,33173,7957,13,46,3860,3285,Malphite
9,2542786996,105,4,14,False,9,7,5,116276,12433,15,180,1052,3152,Fizz
